# Imports

In [64]:
# imports
import numpy as np
from tueplots import bundles, figsizes
import wandb
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd


import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from analysis import sweep2df, plot_typography, stats2string


In [3]:
USETEX = True

In [4]:
plt.rcParams.update(bundles.neurips2022(usetex=USETEX))
# plt.rcParams.update({
#     'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
#                             r'\usepackage{amsmath}'] # boldsymbol
# })

In [5]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)

In [6]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "lti-ica"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

## Max variability

In [7]:
SWEEP_ID = "6u3mgtpz"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_{SWEEP_ID}"
df_max_var, train_log_likelihood_max_var, train_mcc_max_var, val_log_likelihood_max_var, val_mcc_max_var = sweep2df(sweep.runs, filename, save=True, load=True)

### Max variability 10 dimensions

In [11]:
SWEEP_ID = "woiubqya"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_10_{SWEEP_ID}"
df_max_var_10, train_log_likelihood_max_var_10, train_mcc_max_var_10, val_log_likelihood_max_var_10, val_mcc_max_var_10 = sweep2df(sweep.runs, filename, save=True, load=False)

### Rerun for 8 dimensions

In [14]:
SWEEP_ID = "f2n0z65l"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_8_{SWEEP_ID}"
df_max_var_8, train_log_likelihood_max_var_8, train_mcc_max_var_8, val_log_likelihood_max_var_8, val_mcc_max_var_8 = sweep2df(sweep.runs, filename, save=True, load=False)

### Concatenate

In [146]:
df_max_var_concat = pd.concat([df_max_var, df_max_var_8, df_max_var_10])

In [147]:
df_max_var_concat.groupby(["num_comp", "zero_means", "use_B", "use_C"]).mean()[["train_mcc", "max_train_mcc", "val_mcc", "max_val_mcc"]]

,,,,train_mcc,max_train_mcc,val_mcc,max_val_mcc
num_comp,zero_means,use_B,use_C,,,,
2,False,True,True,0.967982,0.987145,0.863717,0.872004
3,False,True,True,0.999989,0.999995,0.997175,0.999113
5,False,True,True,0.995257,0.997676,0.824089,0.826335
8,False,True,True,0.976543,0.983045,0.435802,0.435802
10,False,True,True,0.995729,0.996371,0.732558,0.734933


In [148]:
df_max_var_concat_dict = {comp: df_max_var_concat[df_max_var_concat.num_comp == comp].groupby(["num_comp", "zero_means", "use_B", "use_C"]) for
 comp in sorted(df_max_var_concat.num_comp.unique())}

In [149]:
max_var_stats = [stats2string(df) for comp, df in df_max_var_concat_dict.items()]

## Minimal segments

### Original sweep

In [17]:
SWEEP_ID =  "shrjtedq" #"03w02539"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_{SWEEP_ID}"
df_min_segment, train_log_likelihood_min_segment, train_mcc_min_segment, val_log_likelihood_min_segment, val_mcc_min_segment = sweep2df(sweep.runs, filename, save=True, load=False)

### Rerun for 8 and 10 dimensions

In [75]:
SWEEP_ID = "dvn24tw0"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_8_10_{SWEEP_ID}"
df_min_segment_8_10, train_log_likelihood_min_segment_8_10, train_mcc_min_segment_8_10, val_log_likelihood_min_segment_8_10, val_mcc_min_segment_8_10 = sweep2df(sweep.runs, filename, save=True, load=False)

Encountered a faulty run with ID upbeat-sweep-15


### Concatenate

In [141]:
df_min_segment_concat = pd.concat([df_min_segment, df_min_segment_8_10])

In [142]:
df_min_segment_concat.groupby(["num_comp", "zero_means", "use_B", "use_C"]).mean()[
    ["train_mcc", "max_train_mcc", "val_mcc", "max_val_mcc"]]

train_mcc  max_train_mcc   val_mcc  \
num_comp zero_means use_B use_C                                       
2        False      False False   0.866319       0.911332  0.582510   
                          True    0.766681       0.820678  0.511342   
                    True  False   0.729862       0.826553  0.516959   
                          True    0.697367       0.888114  0.596271   
         True       False False   0.632901       0.847892  0.619947   
                          True    0.674872       0.830931  0.680293   
                    True  False   0.734144       0.842189  0.693170   
                          True    0.725062       0.875474  0.732886   
3        False      False False   0.900751       0.941721  0.741451   
                          True    0.910463       0.941608  0.748249   
                    True  False   0.916051       0.950179  0.807421   
                          True    0.861379       0.911330  0.791572   
         True       False False   0.659496       0.784489  0.633083   
                          True    0.618180       0.768632  0.617080   
                    True  False   0.632505       0.773181  0.603016   
                          True    0.666946       0.804523  0.665514   
5        False      False False   0.891506       0.931900  0.779311   
                          True    0.929386       0.945945  0.804987   
                    True  False   0.927742       0.948141  0.814139   
                          True    0.911213       0.932335  0.744684   
         True       False False   0.657129       0.773772  0.625975   
                          True    0.618422       0.720925  0.581564   
                    True  False   0.620276       0.721762  0.586690   
                          True    0.538554       0.677165  0.480110   
8        False      False False   0.942748       0.957206  0.822469   
                          True    0.939953       0.954366  0.797645   
                    True  False   0.940633       0.952825  0.814091   
         True       False False   0.584651       0.696241  0.559864   
                          True    0.478749       0.636816  0.469697   
                    True  False   0.522868       0.601093  0.466580   
10       False      False True    0.914905       0.943731  0.794419   
                    True  False   0.924568       0.942800  0.771674   
         True       False False   0.731828       0.742926  0.660218   
                          True    0.610552       0.610689  0.490601   
                    True  False   0.604052       0.633980  0.498515   

                                 max_val_mcc  
num_comp zero_means use_B use_C               
2        False      False False     0.619459  
                          True      0.524911  
                    True  False     0.553255  
                          True      0.629053  
         True       False False     0.667954  
                          True      0.717334  
                    True  False     0.708631  
                          True      0.753187  
3        False      False False     0.747056  
                          True      0.760785  
                    True  False     0.822710  
                          True      0.792605  
         True       False False     0.712398  
                          True      0.623745  
                    True  False     0.646789  
                          True      0.696059  
5        False      False False     0.783789  
                          True      0.805552  
                    True  False     0.814934  
                          True      0.747111  
         True       False False     0.654807  
                          True      0.608449  
                    True  False     0.614594  
                          True      0.597913  
8        False      False False     0.822888  
                          True      0.798323  
                    True  False     0.814647  
         True       False False     0.6310

In [143]:
df_min_segment_concat_dict = {
    comp: df_min_segment_concat[df_min_segment_concat.num_comp == comp].groupby(["num_comp", "zero_means", "use_B", "use_C"])
    for
    comp in sorted(df_min_segment_concat.num_comp.unique())}

In [144]:
min_segment_stats = [stats2string(df) for comp, df in df_min_segment_concat_dict.items()]

# Render text for table

In [154]:
all_stats = ["".join(["-----", str(comp),"\n" , min_segment, max_var, "\n"]) for comp, min_segment, max_var in zip(sorted(df_min_segment_concat.num_comp.unique()), min_segment_stats, max_var_stats)]

In [155]:
print("\n".join(all_stats))

-----2
$0.866\scriptscriptstyle\pm 0.033$ & $0.767\scriptscriptstyle\pm 0.158$ & $0.730\scriptscriptstyle\pm 0.191$ & $0.697\scriptscriptstyle\pm 0.090$ & $0.633\scriptscriptstyle\pm 0.104$ & $0.675\scriptscriptstyle\pm 0.132$ & $0.734\scriptscriptstyle\pm 0.158$ & $0.725\scriptscriptstyle\pm 0.115$ & $0.968\scriptscriptstyle\pm 0.055$ & 

-----3
$0.901\scriptscriptstyle\pm 0.054$ & $0.910\scriptscriptstyle\pm 0.061$ & $0.916\scriptscriptstyle\pm 0.044$ & $0.861\scriptscriptstyle\pm 0.062$ & $0.659\scriptscriptstyle\pm 0.201$ & $0.618\scriptscriptstyle\pm 0.241$ & $0.633\scriptscriptstyle\pm 0.110$ & $0.667\scriptscriptstyle\pm 0.109$ & $1.000\scriptscriptstyle\pm 0.000$ & 

-----5
$0.892\scriptscriptstyle\pm 0.057$ & $0.929\scriptscriptstyle\pm 0.025$ & $0.928\scriptscriptstyle\pm 0.026$ & $0.911\scriptscriptstyle\pm 0.045$ & $0.657\scriptscriptstyle\pm 0.116$ & $0.618\scriptscriptstyle\pm 0.079$ & $0.620\scriptscriptstyle\pm 0.025$ & $0.539\scriptscriptstyle\pm 0.078$ & $0.995\script